In [1]:
import numpy as np

In [2]:
# Sample data
expected_returns = np.array([0.12, 0.10, 0.15])  # Example returns for 3 assets
cov_matrix = np.array([[0.1, 0.02, 0.04], [0.02, 0.08, 0.01], [0.04, 0.01, 0.12]])  # Example covariance

def calculate_portfolio_return(weights, returns):
    return np.dot(weights, returns) # Calculate the expected return of the portfolio

def calculate_portfolio_risk(weights, cov_matrix):
    return np.sqrt(np.dot(weights.T, np.dot(cov_matrix, weights))) # Calculate the risk (standard deviation) of the portfolio

def calculate_sharpe_ratio(weights, returns, cov_matrix, risk_free_rate=0.0):
    portfolio_return = calculate_portfolio_return(weights, returns)
    portfolio_risk = calculate_portfolio_risk(weights, cov_matrix)
    return (portfolio_return - risk_free_rate) / portfolio_risk #Calculate the Sharpe Ratio of the portfolio (objective)

# CE method
def cross_entropy_optimization(returns, cov_matrix, num_samples=1000, num_iterations=100, elite_frac=0.2):
    num_assets = len(returns)
    # Initialize weights uniformly
    weights_distribution = np.full((num_assets, 2), (1 / num_assets, 0.1)) # mean and std for each asset

    for _ in range(num_iterations):
        # Sample portfolio weights
        samples = np.array([np.random.normal(loc=weights_distribution[i, 0], scale=weights_distribution[i, 1], size=num_samples) for i in range(num_assets)]).T
        samples = np.abs(samples) / np.sum(np.abs(samples), axis=1, keepdims=True)  # Ensure weights are positive and sum to 1

        # Calculate Sharpe ratio for each sample
        sharpe_ratios = np.array([calculate_sharpe_ratio(sample, returns, cov_matrix) for sample in samples])

        # Select elite samples
        elite_samples = samples[sharpe_ratios.argsort()[-int(num_samples * elite_frac):]]

        # Update weights distribution
        weights_distribution[:, 0] = np.mean(elite_samples, axis=0)  # Update means
        weights_distribution[:, 1] = np.std(elite_samples, axis=0)   # Update stds

    return weights_distribution[:, 0]

# Apply CE method
optimized_weights = cross_entropy_optimization(expected_returns, cov_matrix)
print("Optimized Portfolio Weights:", optimized_weights)


Optimized Portfolio Weights: [0.24257426 0.38118812 0.37623762]


In [3]:
# Sample data: Distances between cities
distances = {
    (0, 1): 10, (0, 2): 15, (0, 3): 20,
    (1, 0): 10, (1, 2): 35, (1, 3): 25,
    (2, 0): 15, (2, 1): 35, (2, 3): 30,
    (3, 0): 20, (3, 1): 25, (3, 2): 30
}
cities = [0, 1, 2, 3]

# Function to calculate the total distance of a given route
def calculate_total_distance(route, distance_matrix):
    """
    Calculate the total distance of a given route based on the distance matrix.
    :param route: List of cities representing the route.
    :param distance_matrix: Matrix containing distances between each pair of cities.
    :return: Total distance of the route.
    """
    total_distance = 0
    for i in range(len(route)):
        # Sum distances between consecutive cities, wrapping back to the start
        total_distance += distance_matrix[route[i], route[(i + 1) % len(route)]]
    return total_distance

# CE method
def cross_entropy_method_tsp(distances, num_cities, num_samples, iterations):
    """
    Solves the Traveling Salesman Problem using the Cross-Entropy method.
    :param distances: Dict with tuple keys representing city pairs and values representing distances.
    :param num_cities: Number of cities in the TSP.
    :param num_samples: Number of samples to generate per iteration.
    :param iterations: Number of iterations to run the algorithm.
    :return: Best route found and its distance.
    """
    # Convert the distance dictionary to a matrix for easy lookup
    distance_matrix = np.zeros((num_cities, num_cities))
    for (i, j), dist in distances.items():
        distance_matrix[i, j] = dist

    # Initialize a probability matrix for selecting the next city
    prob_matrix = np.full((num_cities, num_cities), 1.0 / num_cities)

    best_route = None
    best_distance = float('inf')

    for _ in range(iterations):
        # Generate a set of routes based on the current probability matrix
        routes = []
        for _ in range(num_samples):
            route = [0]  # Start from the first city
            while len(route) < num_cities:
                # Choose the next city based on probabilities, excluding already visited cities
                current_city = route[-1]
                next_cities = list(set(range(num_cities)) - set(route))
                next_city_probs = prob_matrix[current_city, next_cities]
                next_city_probs /= next_city_probs.sum()  # Normalize probabilities
                next_city = np.random.choice(next_cities, p=next_city_probs)
                route.append(next_city)
            routes.append(route + [route[0]])  # Complete the route by returning to the start

        # Evaluate each route to find the total distance
        distances = np.array([calculate_total_distance(route, distance_matrix) for route in routes])
        # Identify the top routes with the shortest distances
        top_indices = distances.argsort()[:num_samples // 2]
        top_routes = np.array(routes)[top_indices]

        # Update the probability matrix based on the successful routes
        prob_matrix.fill(0)
        for route in top_routes:
            for i in range(num_cities):
                prob_matrix[route[i], route[(i + 1) % num_cities]] += 1
        prob_matrix /= prob_matrix.sum(axis=1, keepdims=True)  # Ensure each row sums to 1

        # Check if the best route so far is found in this iteration
        current_best_index = top_indices[0]
        current_best_distance = distances[current_best_index]
        if current_best_distance < best_distance:
            best_route = routes[current_best_index]
            best_distance = current_best_distance

    return best_route, best_distance


# Parameters
num_cities = len(cities)
num_samples = 100
iterations = 100

# Find the best route
best_route, best_distance = cross_entropy_method_tsp(distances, num_cities, num_samples, iterations)
print("Best Route:", best_route)
print("Total Distance:", best_distance)


Best Route: [0, 2, 3, 1, 0]
Total Distance: 80.0
